In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("./data/salary/train.csv")

In [2]:
#change obj column to num
def to_num(df):
    col_name = df.columns.values
    obj_list = []
    for name in col_name:
        if df[name].dtype == 'object':
            obj_list.append(name)
    obj_dict = {}
    for name in obj_list:#list of object column
        keys = list(dict(list(df.groupby(name))).keys())#various value of this column
        obj_dict[name] = keys
        for i in range(len(keys)):
            key = keys[i]
            df_col = df[name]
            df_col[df[name] == key] = i
    return df,obj_dict
# change num converted from obj to one-hot
def sperate_obj(df):
    col_name = df.columns.values
    obj_list = []
    for name in col_name:
        if df[name].dtype == 'object':
            obj_list.append(name)
    obj_dict = {}
    for name in obj_list:#list of object column
        keys = list(dict(list(df.groupby(name))).keys())#various value of this column
        
        if len(keys) <= 2 :#value types no more than 2,change previous column only
            obj_dict[name] = keys
            df_col = df[name]
            df_col[df[name] == keys[0]] = 0
            df_col[df[name] != 0] = 1
        else:#need convert to on-hot,i.e.need add and drop columns
            obj_dict[name] = []
            for key in keys:
                #add new col
                col_name = name+"_"+key
                df[col_name] = 0
                new_col = df[col_name]
                new_col[df[name] == key] = 1
                obj_dict[name].append(col_name)
                
            #drop col
            df.drop(name,axis = 1,inplace=True)
        
    return df,obj_dict

In [3]:
df_convert,obj_dict = sperate_obj(df)

C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
df_convert.to_csv("./result/salary/test.csv")

In [ ]:
f = open("./result/salary/test",'w')
f.writelines(str(obj_dict))
f.close()

In [5]:
df_label = df_convert.loc[:,'income']
df_train = df_convert.drop(labels=['income'],axis=1)

In [6]:
data_y = df_label.values.reshape((1,-1)).astype(np.float) #(1,n)
#must use astype(float),otherwise the dtype is object,will err in numpy operation
data_x = df_train.values.astype(np.float) .T #(dim,n)
#feature scaling
data_x = (data_x - np.mean(data_x,axis=1).reshape((-1,1)))/np.std(data_x,axis=1).reshape((-1,1))

In [12]:
#must pay attention : instead of mul(w,x) + b, is sum(mul(w,x)) + b
dim,n = data_x.shape
w = np.random.normal(0,0.02,(dim,1))
#w = np.ones((dim,1))
b = np.zeros((1,1))
lr = 0.3
epoches = 2000

def shuffle(vali_ratio = 0.2):
    seed = np.random.randint(0,1e6)

    np.random.seed(seed)
    np.random.shuffle(data_x)
    np.random.seed(seed)
    np.random.shuffle(data_y)
    
    return data_x,data_y

def sigmoid(x):
    return 1./(1.+ np.exp(-x))

def f(x,w,b):
    return sigmoid(np.matmul(w.T,x) + b)

def loss(x,y):
    pred = f(x,w,b)
    pred_log = np.log(pred)#q(1) = f(x)
    no_pred_log = np.log(1.0-pred)#q(0) = 1 - f(x) ; f(x) : probability of class A(1)
    loss_val = -1.* (np.matmul(y,pred_log.T) + np.matmul((1.0-y),no_pred_log.T))
    return loss_val

def accuarcy(x,y):
    pred =f(x,w,b)
    pred = np.rint(pred)
    return np.mean(np.equal(pred,y).astype('float'))
    
w_g_square_sum = np.zeros((dim,1))
b_g_square_sum = np.zeros((1,1))
for i in range(epoches):
    #if this program is interrupted,and this shuffle function is not end,the data wiil wrong
    #the data_x and data_y may not match again...
    #x,y = shuffle() 
    
    x = data_x
    y = data_y
   
    w_g = np.matmul(-x, (y - f(x,w,b)).T)#(1,dim)
    b_g = np.dot(-np.ones((1,n)),(y - f(x,w,b)).T)#(1,1)
    
    w_g_square_sum += w_g ** 2
    b_g_square_sum += b_g ** 2
    
    w1 = w - (lr / np.sqrt(w_g_square_sum))*(w_g)
    b1 = b - (lr / np.sqrt(b_g_square_sum))*(b_g)
    
    w = w1
    b = b1
    if i % 10 == 0 :
        print("loss:{},acc:{}".format(loss(x,y),accuarcy(x,y)))

C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in matmul


loss:[[nan]],acc:0.7339762292312889
loss:[[10899.15003055]],acc:0.8498203372132306
loss:[[10483.02651867]],acc:0.8535671508860293
loss:[[10387.29293272]],acc:0.8543042289855962
loss:[[10347.24973267]],acc:0.8543349405730782
loss:[[10326.75743632]],acc:0.8542428058106324
loss:[[10314.98463587]],acc:0.853751420410921
loss:[[10307.59344614]],acc:0.8537207088234391
loss:[[10302.62986567]],acc:0.8535671508860293
loss:[[10299.1489776]],acc:0.8535057277110654
loss:[[10296.63961135]],acc:0.8534443045361014
loss:[[10294.77608452]],acc:0.8535057277110654
loss:[[10293.33958913]],acc:0.8534443045361014
loss:[[10292.18901396]],acc:0.8534135929486195
loss:[[10291.23588149]],acc:0.8533521697736556
loss:[[10290.42450734]],acc:0.8533214581861737
loss:[[10289.71911442]],acc:0.8533521697736556
loss:[[10289.09608026]],acc:0.8532907465986916
loss:[[10288.53932965]],acc:0.8533214581861737
loss:[[10288.03756677]],acc:0.8533214581861737
loss:[[10287.58258407]],acc:0.8532907465986916
loss:[[10287.16821094]],ac

In [ ]:
f = open("./result/salary/wb",'w')
f.writelines(str(w).replace('  ',','))
f.writelines(str(b))
f.close()

In [13]:
def load_test(df,obj_dict):
    col_name = df.columns.values
    obj_list = []
    for name in col_name:
        if df[name].dtype == 'object':
            obj_list.append(name)
   
    for name in obj_list:#list of object column
        keys = list(obj_dict[name])#various value of this column
        
        if len(keys) <= 2 :#value types no more than 2,change previous column only;no prefix
            df_col = df[name]
            df_col[df[name] == keys[0]] = 0
            df_col[df[name] != 0] = 1
        else:#need convert to on-hot,i.e.need add and drop columns
            keys = [key[len(name)+1:] for key in keys]#has prefix
            for key in keys:
                #add new col
                col_name = name+"_"+key
                df[col_name] = 0
                new_col = df[col_name]
                new_col[df[name] == key] = 1
                obj_dict[name].append(col_name)
                
            #drop col
            df.drop(name,axis = 1,inplace=True)
        
    return df

df_t = pd.read_csv("./data/salary/test.csv",encoding='Big5')
df_t_c = load_test(df_t,obj_dict)

C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
df_t_c.to_csv("./result/salary/temp.csv")

In [19]:
test_x = df_t_c.values.astype('float').T #(dim,n)
#remember feature scaling
test_x = (test_x - np.mean(test_x,axis=1).reshape((-1,1)))/(np.std(test_x,axis=1)+ 1e-10).reshape((-1,1))

pred = f(test_x,w,b)
pred = np.rint(pred).reshape((-1,))#<=50K : 0; >50K : 1

pred_list = list(pred)
id_list = range(1,len(pred)+1)
df_out = pd.DataFrame({'id':id_list,'label':pred_list},dtype='int')

In [20]:
df_out.to_csv("./result/salary/pred.csv",index=False)

In [18]:
test_x - np.mean(test_x,axis=0)


(16281,)